In [ ]:
! pip install kaggle

In [ ]:
from google.colab import files

# Kaggle API anahtarını yükleyin attığım kaggle.json dosyasını
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d sachinkumar413/covid-pneumonia-normal-chest-xray-images # train dataseti
!kaggle datasets download -d pranavraikokte/covid19-image-dataset # Validation ve 1.test
!kaggle datasets download -d amanullahasraf/covid19-pneumonia-normal-chest-xray-pa-dataset #2. test

 95% 263M/277M [00:02<00:00, 166MB/s]
100% 277M/277M [00:02<00:00, 130MB/s]
 94% 148M/158M [00:01<00:00, 136MB/s]
100% 158M/158M [00:01<00:00, 138MB/s]
100% 1.91G/1.91G [00:19<00:00, 140MB/s]
100% 1.91G/1.91G [00:19<00:00, 104MB/s]


In [ ]:
import zipfile
import os


zip_path = '/content/covid-pneumonia-normal-chest-xray-images.zip'

# Çıkartılacak dizini belirtin
extracted_folder_path = '/content/covid-pneumonia-normal-chest-xray-images'

# Zip dosyasını çıkart
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

# Çıkartılan dosyaları göster
os.listdir(extracted_folder_path)

zip_path = '/content/covid19-image-dataset.zip'

# Çıkartılacak dizini belirtin
extracted_folder_path = '/content/covid19-image-dataset'

# Zip dosyasını çıkart
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

# Çıkartılan dosyaları göster
os.listdir(extracted_folder_path)


zip_path = '/content/covid19-pneumonia-normal-chest-xray-pa-dataset.zip'

# Çıkartılacak dizini belirtin
extracted_folder_path = '/content/covid19-pneumonia-normal-chest-xray-pa-dataset'

# Zip dosyasını çıkart
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

# Çıkartılan dosyaları göster
os.listdir(extracted_folder_path)

['normal', 'covid', 'pneumonia']

In [ ]:
!pip install split_folders

In [ ]:
import splitfolders
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2,ResNet152V2,Xception
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
import os

In [ ]:
# Define the paths to your training, validation and test datasets
training_dir = '/content/covid-pneumonia-normal-chest-xray-images'
validation_dir = '/content/covid19-image-dataset/Covid19-dataset/test'


In [ ]:
# Define data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    brightness_range=[0.8, 1.2],
    width_shift_range=0.2,
    rotation_range=20,
    horizontal_flip=True,
    fill_mode='nearest'
)


# Load your dataset
train_generator = train_datagen.flow_from_directory(
    training_dir,
    target_size=(320, 320),
    batch_size=32,
    shuffle = True,
    class_mode='categorical'
)

val_datagen = ImageDataGenerator(
    rescale=1./255
    )
val_generator = val_datagen.flow_from_directory(
    validation_dir,
    target_size=(320, 320),
    batch_size=32,
    shuffle = True,
    class_mode='categorical'
)





Found 5228 images belonging to 3 classes.
Found 66 images belonging to 3 classes.


In [ ]:
# Model Architecture
# TODO: Dediğim modelleri buraya import edip yerine yazın her şey çalışacaktır!
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(320, 320, 3))

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(3, activation='softmax')(x)  # Adjust NUM_CLASSES to your task

model = Model(inputs=base_model.input, outputs=predictions)

base_model2= ResNet152V2(weights='imagenet', include_top=False, input_shape=(320, 320, 3))

for layer in base_model2.layers:
    layer.trainable = False

x = base_model2.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(3, activation='softmax')(x)  # Adjust NUM_CLASSES to your task
model2 = Model(inputs=base_model2.input, outputs=predictions)

base_model3= Xception(weights='imagenet', include_top=False, input_shape=(320, 320, 3))

for layer in base_model3.layers:
    layer.trainable = False

x = base_model3.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(3, activation='softmax')(x)  # Adjust NUM_CLASSES to your task
model3 = Model(inputs=base_model3.input, outputs=predictions)

models = [model, model2,model3]
model_input = keras.Input(shape=(320, 320, 3))
model_outputs = [model(model_input) for model in models]
ensemble_output = keras.layers.Average()(model_outputs)
ensemble_model = keras.Model(inputs=model_input, outputs=ensemble_output)



In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
ensemble_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:


ensemble_model.fit(
    train_generator,
    epochs=5,
    validation_data=val_generator
)

Epoch 1/5
164/164 [==============================] - 234s 1s/step - loss: 0.4243 - accuracy: 0.8428 - val_loss: 0.2246 - val_accuracy: 0.8636
Epoch 2/5
164/164 [==============================] - 198s 1s/step - loss: 0.2479 - accuracy: 0.9212 - val_loss: 0.2160 - val_accuracy: 0.9242
Epoch 3/5
164/164 [==============================] - 201s 1s/step - loss: 0.2466 - accuracy: 0.9225 - val_loss: 0.3455 - val_accuracy: 0.8939
Epoch 4/5
164/164 [==============================] - 199s 1s/step - loss: 0.2029 - accuracy: 0.9365 - val_loss: 0.1803 - val_accuracy: 0.9242
Epoch 5/5
164/164 [==============================] - 199s 1s/step - loss: 0.1964 - accuracy: 0.9394 - val_loss: 0.2371 - val_accuracy: 0.9242


In [ ]:
## Fine tuning to improve accuracy
import keras

fine_tune_at = 125
# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False


custom_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)

# Compile the model
ensemble_model.compile(optimizer=custom_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = keras.callbacks.EarlyStopping( monitor='val_accuracy',patience=5,restore_best_weights=True)

# Train the model with fine-tuning
history = ensemble_model.fit(
    train_generator,
    epochs=5,
    validation_data=val_generator,
    callbacks= [early_stopping]
    )

Epoch 1/5
164/164 [==============================] - 221s 1s/step - loss: 0.1441 - accuracy: 0.9570 - val_loss: 0.1819 - val_accuracy: 0.9242
Epoch 2/5
164/164 [==============================] - 196s 1s/step - loss: 0.1463 - accuracy: 0.9554 - val_loss: 0.1577 - val_accuracy: 0.9697
Epoch 3/5
164/164 [==============================] - 198s 1s/step - loss: 0.1410 - accuracy: 0.9570 - val_loss: 0.1615 - val_accuracy: 0.9242
Epoch 4/5
164/164 [==============================] - 198s 1s/step - loss: 0.1339 - accuracy: 0.9602 - val_loss: 0.1372 - val_accuracy: 0.9848
Epoch 5/5
164/164 [==============================] - 197s 1s/step - loss: 0.1279 - accuracy: 0.9623 - val_loss: 0.1560 - val_accuracy: 0.9242


In [ ]:
test_datagen = ImageDataGenerator(
    rescale=1./255
    )
test_generator = test_datagen.flow_from_directory(
    "/content/covid19-image-dataset/Covid19-dataset/train",
    target_size=(320, 320),
    batch_size=32,
    shuffle = True,
    class_mode='categorical'
)

test_datagen2 = ImageDataGenerator(
    rescale=1./255
    )
test_generator2 = test_datagen.flow_from_directory(
    "/content/covid19-pneumonia-normal-chest-xray-pa-dataset",
    target_size=(320, 320),
    batch_size=32,
    shuffle = True,
    class_mode='categorical'
)

In [ ]:
ensemble_model.evaluate(test_generator)

8/8 [==============================] - 9s 1s/step - loss: 0.2500 - accuracy: 0.9323


[0.2500132620334625, 0.9322709441184998]

In [ ]:
ensemble_model.evaluate(test_generator2)

216/216 [==============================] - 164s 758ms/step - loss: 2.9638 - accuracy: 0.6739


[2.9637863636016846, 0.673862636089325]